In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [31]:
from langchain_community.chat_models.sambanova import ChatSambaNovaCloud
from langchain_core.rate_limiters import InMemoryRateLimiter

from struct_vs_unstruct.helpers.rate import AmongProcessRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.166,  # <-- Can only make a request once every 10 seconds!!
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=50,  # Controls the maximum burst size.
)

llm = ChatSambaNovaCloud(
    model="llama3-405b", max_tokens=1024, temperature=0.7, top_k=1, top_p=0.01, rate_limiter=rate_limiter
)

In [32]:
def map_fn(instance, idx):
    from typing import Optional, TypedDict
    from langgraph.graph import END, START, StateGraph
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.prompts import PromptTemplate
    
    class State(TypedDict):
        start: int
        end: int
        response: Optional[str]
    
    def call(inputs):
        # Initialize everything inside the process
        parser = StrOutputParser()
        pt = PromptTemplate.from_template("Count from {start} to {end}")
        
        # Build the chain inside the process
        chain = pt | llm | parser
    
        # Call the chain and pass results to the queue
        return {
            "response": chain.invoke(inputs)
        }
    graph = StateGraph(State)
    graph.add_node("call", call)
    graph.add_edge(START, "call")
    graph.add_edge("call", END)
    
    
    app = graph.compile()
    result = app.invoke({"start": idx, "end": idx+20})

    print(result)

In [26]:
from datasets import load_dataset
ds = load_dataset("sachithgunasekara/t4d")

In [ ]:
import multiprocessing
ds.map(map_fn, with_indices=True)

Map:   0%|          | 0/564 [00:00<?, ? examples/s]

{'start': 0, 'end': 20, 'response': "Here's the count from 0 to 20:\n\n0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20."}
{'start': 1, 'end': 21, 'response': "Here's the count from 1 to 21:\n\n1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21."}
{'start': 2, 'end': 22, 'response': "Here's the count from 2 to 22:\n\n2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22."}
